In [3]:
import pandas as pd
import data_util
import numpy as np

import torch
from transformers import BertTokenizer
from keras_preprocessing.sequence import pad_sequences
# from transformers import BertForSequnceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [4]:
train_data, test_data = data_util.load_data()

reading data


In [5]:
from konlpy.tag import Komoran
from textrank import KeywordSummarizer
komoran = Komoran

In [9]:
def komoran_tokenizer(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

In [62]:
temp_string = train_data['document'][57]
words = temp_string.split(' ')

In [63]:
train_data['document'][57]

'너무 충격적이엇다 기분을 완전히 푹 꺼지게 하는 느낌 활력이라고는 하나도 없는 너무나도 무거운지독하고 차갑고 무자비하다 그저 일본인들의 상상력은 정말 대단한거 같다는 생각이 든다'

In [64]:
words

['너무',
 '충격적이엇다',
 '기분을',
 '완전히',
 '푹',
 '꺼지게',
 '하는',
 '느낌',
 '활력이라고는',
 '하나도',
 '없는',
 '너무나도',
 '무거운지독하고',
 '차갑고',
 '무자비하다',
 '그저',
 '일본인들의',
 '상상력은',
 '정말',
 '대단한거',
 '같다는',
 '생각이',
 '든다']

In [65]:
from textrank import KeywordSummarizer

summarizer = KeywordSummarizer(tokenize=komoran_tokenizer, min_count=1, min_cooccurrence=1)
summa_words = summarizer.summarize(words, topk=30)
print(summa_words)

[('독하/VA', 1.0), ('무겁/VA', 1.0), ('거/NNB', 1.0), ('대단/XR', 1.0), ('기분/NNG', 0.15000000000000002), ('꺼지/VV', 0.15000000000000002), ('하/VV', 0.15000000000000002), ('느낌/NNG', 0.15000000000000002), ('활력/NNG', 0.15000000000000002), ('없/VA', 0.15000000000000002), ('들/VV', 0.15000000000000002), ('생각/NNG', 0.15000000000000002), ('차갑/VA', 0.15000000000000002), ('자비/NNG', 0.15000000000000002), ('일본인/NNG', 0.15000000000000002), ('상상력/NNP', 0.15000000000000002), ('같/VA', 0.15000000000000002), ('충격/NNG', 0.15000000000000002)]


In [66]:
# temp = summa_words[0][0]
# temp = []
# for 
temp = [summa_words[i][0] for i in range(len(summa_words))]

In [67]:
temp

['독하/VA',
 '무겁/VA',
 '거/NNB',
 '대단/XR',
 '기분/NNG',
 '꺼지/VV',
 '하/VV',
 '느낌/NNG',
 '활력/NNG',
 '없/VA',
 '들/VV',
 '생각/NNG',
 '차갑/VA',
 '자비/NNG',
 '일본인/NNG',
 '상상력/NNP',
 '같/VA',
 '충격/NNG']

In [20]:
temp_string

'사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다'

In [19]:
from textrank import KeywordSummarizer

docs = ['list of str form', 'sentence list']

keyword_extractor = KeywordSummarizer(
    tokenize = lambda x:x.split(),      # YOUR TOKENIZER
    window = -1,
    verbose = False
)

sents = temp_string

keywords = keyword_extractor.summarize(sents, topk=30)
for word, rank in keywords:
    print(word)
    print(rank)

였
0.15000000000000002
가
0.15000000000000002
기
0.15000000000000002
던
0.15000000000000002
보
0.15000000000000002
스
0.15000000000000002
이
0.15000000000000002


In [5]:
train_data, val_data = np.split(train_data.sample(frac=1), [int(0.8*len(train_data))])

In [10]:
def tokenize(tokenizer, sentences, MAX_LEN):

    input_ids = []

    for i, sent in enumerate(sentences):
        if (i + 1) % 1000 == 0:
            print("tokenizing " + str(i + 1) + " out of " + str(len(sentences)))
        encoded_sent = tokenizer.encode(
                sent,
                add_special_tokens=True,
                max_length = 128, # 32 / 64 / 128 / none
                pad_to_max_length = True
                )
        input_ids.append(encoded_sent)

    #  pdb.set_trace()

    MAX_LEN = max([len(sen) for sen in input_ids])

    print("padding...")
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
    
    attention_masks = []
    for sent in input_ids:
        att_mask = [int(token_id > 0) for token_id in sent]
        attention_masks.append(att_mask)

    return input_ids, MAX_LEN, attention_masks

In [42]:
#  50 - posi
#  57 - nega
train_data[50:60]

,id,document,label
50,9063648,영화가 사람의 영혼을 어루만져 줄 수도 있군요 거친 세상사를 잠시 잊고 동화같은 영...,1
51,8272095,야 세르게이 작은고추의 매운맛을 보여주마 포퐁저그 콩진호가 간다,0
52,2345905,이렇게 가슴시리게 본 드라마가 또 있을까 감동 그 자체,1
53,7865630,난또 저 꼬마애가 무슨 원한이 깊길래 했더니 그냥 혼자 나대다 걸 어쩌라고,0
54,7207064,재미있어요,1
55,5719655,전 좋아요,1
56,1651126,최고,0
57,7246040,너무 충격적이엇다 기분을 완전히 푹 꺼지게 하는 느낌 활력이라고는 하나도 없는 너무...,1
58,717775,심심한영화,0
59,8317483,백봉기 언제나오나요,1


In [16]:
MAX_LEN = 64
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

train_inputs, train_MAX_LEN, train_attn_masks = tokenize(tokenizer, train_data['document'],MAX_LEN)
train_labels = train_data['label'].values

tokenizing 1000 out of 116632
tokenizing 2000 out of 116632
tokenizing 3000 out of 116632
tokenizing 4000 out of 116632
tokenizing 5000 out of 116632
tokenizing 6000 out of 116632
tokenizing 7000 out of 116632
tokenizing 8000 out of 116632
tokenizing 9000 out of 116632
tokenizing 10000 out of 116632
tokenizing 11000 out of 116632
tokenizing 12000 out of 116632
tokenizing 13000 out of 116632
tokenizing 14000 out of 116632
tokenizing 15000 out of 116632
tokenizing 16000 out of 116632
tokenizing 17000 out of 116632
tokenizing 18000 out of 116632
tokenizing 19000 out of 116632
tokenizing 20000 out of 116632
tokenizing 21000 out of 116632
tokenizing 22000 out of 116632
tokenizing 23000 out of 116632
tokenizing 24000 out of 116632
tokenizing 25000 out of 116632
tokenizing 26000 out of 116632
tokenizing 27000 out of 116632
tokenizing 28000 out of 116632
tokenizing 29000 out of 116632
tokenizing 30000 out of 116632
tokenizing 31000 out of 116632
tokenizing 32000 out of 116632
tokenizing 33000 

In [17]:
train_inputs[:5]

array([[  101,   100,  1463, 30019, 29991, 30008,   100,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [18]:
train_labels[:5]

array([1, 1, 0, 0, 0])

In [ ]:
['사이몬페그의',
 '익살스런',
 '연기가',
 '돋보였던',
 '영화스파이더맨에서',
 '늙어보이기만',
 '했던',
 '커스틴',
 '던스트가',
 '너무나도',
 '이뻐보였다']

[('티/NNG', 1.0),
 ('커스/NNP', 1.0),
 ('보이/VV', 1.0),
 ('늙/VV', 1.0),
 ('스파이더맨/NNP', 1.0),
 ('영화/NNG', 1.0),
 ('하/VV', 0.15000000000000002),
 ('돋보이/VV', 0.15000000000000002),
 ('연기/NNG', 0.15000000000000002),
 ('익살/NNG', 0.15000000000000002)]